Di sotto ci sono le linee di codice per l'installazione di selenium insieme ad altre componenti, il tutto è stato copiato da Web_Scraping_with_Selenium.ipynb. Il documento usato dai prof nella prima lezione sul web scraping

In [1]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium==4.9.1

Executing: /tmp/apt-key-gpghome.43fwMQHnEr/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.jFmiiwiKD6/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.FtKKQKiCA9/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

Lo script seguente installa alcune componenti da usare per lo scraping

Ecco la descrizione fornita dai prof nel progetto originale
1.   sys: to setup the path of chrome driver
2.   **selenium**: to emulate the user behaviour
3.   **pandas**: to work with data
4.   tqdm: to show a progress bar in our notebook
5.   json: to work with json format

In [2]:
import sys
import logging
from selenium.webdriver.remote.remote_connection import LOGGER
LOGGER.setLevel(logging.WARNING)
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from tqdm.notebook import tqdm
import pandas
import json
import pprint

Dopo impostiamo le impostazioni del ghost browser che andrà ad usare selenium

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("window-size=1900,800")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36")

Sotto abbiamo avviato lo scraper e lo facciamo andare sulla pagina di Knowledge share

In [4]:
wd = webdriver.Chrome('chromedriver', options=chrome_options)

Lo script seguente clicca su "Accetta" nel richiesta sulla privacy dei Cookies. Si noti che nel link è già stato impostato il settore che studieremo: sanità e biomedicina

In [5]:
import time

wd.get("https://www.knowledge-share.eu/area/sanita-e-biomedicale/")
time.sleep(5)

if (len(wd.find_elements(By.CSS_SELECTOR, ".iubenda-cs-accept-btn")) > 0):
  wd.find_elements(By.CSS_SELECTOR, ".iubenda-cs-accept-btn")[0].click()

Facciamo una prova per verificare che andremo a selezionare gli elementi corretti

In [6]:
list_patent= wd.find_elements(By.CSS_SELECTOR,".wrapbrevetto")
print(len(list_patent))

18


Gli script seguenti andranno a comporre il vero processo di scraping. Per cominciare ho usato un paio di linee fornite dai prof a lezione per individuare gli elementi di CSS Selector da utilizzare.

In [7]:
details = []

for item in list_patent:   # list_patent contiene i brevetti #
    url = item.find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href")
    title = item.find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("title")
    description = ""

    if(len(item.find_elements(By.CSS_SELECTOR,".minidesc")) > 0):
      description = item.find_elements(By.CSS_SELECTOR,".minidesc")[0].text

    details.append({'url': url,
                    'title': title,
                    'description': description})
    ### funzione scroll down  ###
    wd.execute_script("window.scrollBy(0,1200)")

Questo script serve a vedere e confermare che lo scraper abbia scaricato le informazioni che ci servono

In [ ]:
pprint.pprint(details[3:])

[{'description': '"Lab on a chip” ottimizzato per studiare il potere invasivo '
                 'delle cellule in sistemi tridimensionali che simulano '
                 'l’ambiente fisiologico.',
  'title': '“3D-cell migration chip”per saggi di migrazione cellulare',
  'url': 'https://www.knowledge-share.eu/brevetto/3d-cell-migration-chipper-saggi-di-migrazione-cellulare/'},
 {'description': 'Variazione genetica denominata CD56-CD16+PerfnegCD7- dalla '
                 'popolazione cellulare NK CD94/NKG2C+ matura, che ammonta a '
                 '4842 geni.',
  'title': '4842 geni identificano la progenie da CD56-CD16+PerfnegCD7-',
  'url': 'https://www.knowledge-share.eu/brevetto/4842-geni-identificano-la-progenie-da-cd56-cd16perfnegcd7/'},
 {'description': 'Innovativo adiuvante vaccinale in grado di indurre una '
                 'risposta immune specifica e duratura coniugandosi '
                 'chimicamente ad un antigene.',
  'title': 'Acido Ialuronico come adiuvante natural

Questa sarà la funzione che andrà sistematicamente a scaricare le informazioni nella pagina dei brevetti. Fornita sempre dai prof, con qualche piccola modifica la possiamo usare per il nostro progetto

In [ ]:
def parse_item(patent):
    url = patent.find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("href")
    title = patent.find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("title")
    description = ""

    if(len(patent.find_elements(By.CSS_SELECTOR,".minidesc")) > 0):
      description = patent.find_elements(By.CSS_SELECTOR,".minidesc")[0].text

    return {'url': url,
            'title': title,
            'description': description}

Queste funzioni andremo ad usarle negli step successivi del processo di scraping. La prima serve a dire al driver di attendere

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

Questa è la forma finale dello scraper. Gestisce il cambio di pagine attraverso la modifica diretta del link alla 5ta riga. Il modo in cui lo fa è attraverso una generazione automatica di numeri, che avviene grazie alla funzione range trovata alla 3za riga.

Naturalmente chiederemo allo scraper di scaricare tutte le pagine, dalla prima alla millesima. Come potete immaginare il sito web non ha mille pagine, per gestire la scoperta di una pagina vuota che non contenga dati è stata inserita la funzione break che automaticamente ferma lo scraper se non vengono trovati elementi .wrapbrevetto ossia, le patenti.

In [ ]:
patents = []

for page in tqdm(range(1,1000)):
  print(f"Download data from page {page}")
  wd.get(f"https://www.knowledge-share.eu/area/sanita-e-biomedicale/?sf_paged={page}")
  time.sleep(10)
  list_patents= wd.find_elements(By.CSS_SELECTOR,".wrapbrevetto")
  if len(list_patents) <= 0:
    break

  for patent in list_patents:
    patents.append(parse_item(patent))
    wd.execute_script("window.scrollBy(0,1200)")
### comando per scendere nella pagina, simulando uno "scroll down"  ###

  0%|          | 0/999 [00:00<?, ?it/s]

Download data from page 1
Download data from page 2
Download data from page 3
Download data from page 4
Download data from page 5
Download data from page 6
Download data from page 7
Download data from page 8
Download data from page 9
Download data from page 10
Download data from page 11
Download data from page 12
Download data from page 13
Download data from page 14
Download data from page 15
Download data from page 16
Download data from page 17
Download data from page 18
Download data from page 19
Download data from page 20
Download data from page 21
Download data from page 22
Download data from page 23
Download data from page 24
Download data from page 25
Download data from page 26
Download data from page 27
Download data from page 28
Download data from page 29
Download data from page 30
Download data from page 31
Download data from page 32
Download data from page 33
Download data from page 34
Download data from page 35
Download data from page 36
Download data from page 37
Download d

In [ ]:
print(len(patents))
pprint.pprint(patents[0:3])

993
[{'description': 'Permette di mantenere in condizioni fisiologiche un organo '
                 'in ambiente extracorporeo, consentendone la valutazione di '
                 'idoneità funzionale.',
  'title': '‘BIOLAB PERFUSOR’ – Dispositivo medico di perfusione meccanica',
  'url': 'https://www.knowledge-share.eu/brevetto/biolab-perfusor-dispositivo-medico-di-perfusione-meccanica/'},
 {'description': 'Miscela di glucocorticoidi, A_Atrofi, costituita da '
                 "modulatori per rallentare l'atrofia muscolare per il "
                 'trattamento della sarcopenia e della cachessia.',
  'title': '“A_Atrofi” – Regolatori della massa muscolare',
  'url': 'https://www.knowledge-share.eu/brevetto/a_atrofi-regolatori-della-massa-muscolare/'},
 {'description': 'Casco in grado di chiamare soccorsi in caso di incidente, e '
                 'grado di stimare i danni subiti dalla testa attraverso un '
                 'modulo di IA.',
  'title': '“Health-Met” – Casco con Intellige

Porta il database di patents a formato csv in modo da poterlo leggere su altri software ETL ma anche per avere una sorta di check-point da cui poter ripartire: se devo spegnere il pc o cancello qualcosa che invece serviva, basta caricare il file delle patenti già scaricate.

In [ ]:
import pandas as pd
ds_patents = pd.DataFrame(patents)
ds_patents.set_index("url")
ds_patents.head()

In [ ]:
ds_patents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   url          993 non-null    object
 1   title        993 non-null    object
 2   description  993 non-null    object
dtypes: object(3)
memory usage: 23.4+ KB


In [ ]:
ds_patents.to_csv('ds_patents.csv')

# Contenuti della pagina

Adesso comincieremo a scaricare le informazione dei brevetti.

Due pagine erano state fatte in modo molto diverso dalle altre, le abbiamo manualmente rimosso in modo da non avere problemi.

Tutte le pagine web vuote andranno inserite nella cartella empty_pages in modo che non inquinino il database, possiamo controllarle in un secondo momento se necessario o farci successivi studi



In [ ]:
ds_new = ds_patents.drop(432)


In [ ]:
ds_patents = ds_new.drop(795)

In [ ]:
ds_patents.to_csv('ds_patents.csv')

In [ ]:
# open csv file
import pandas as pd
ds_patents = pd.read_csv("ds_patents.csv", index_col="Unnamed: 0")
ds_patents.head()

,url,title,description
0,https://www.knowledge-share.eu/brevetto/biolab...,‘BIOLAB PERFUSOR’ – Dispositivo medico di perf...,Permette di mantenere in condizioni fisiologic...
1,https://www.knowledge-share.eu/brevetto/a_atro...,“A_Atrofi” – Regolatori della massa muscolare,"Miscela di glucocorticoidi, A_Atrofi, costitui..."
2,https://www.knowledge-share.eu/brevetto/health...,“Health-Met” – Casco con Intelligenza Artificiale,Casco in grado di chiamare soccorsi in caso di...
3,https://www.knowledge-share.eu/brevetto/3d-cel...,“3D-cell migration chip”per saggi di migrazion...,"""Lab on a chip” ottimizzato per studiare il po..."
4,https://www.knowledge-share.eu/brevetto/4842-g...,4842 geni identificano la progenie da CD56-CD1...,Variazione genetica denominata CD56-CD16+Perfn...


Questo scraper andrà a scaricare le informazioni contenute all'interno delle pagine dei brevetti.
Alcuni brevetti non rendevano disponibili le informazioni, succedeva spesso se erano stati caricati da meno di 18 mesi, questo è il motivo del codice che va dalla riga 37 alla 42

In [ ]:
import time
patent_details = []
anomalies = []
num = 0

### for loop per accedere a ogni link delle pagine web scrapate ###
for patent_id, patent in ds_patents.iterrows():
      num = num + 1
      print(f"patent number {num}/991")
      link = patent["url"]   #link dei brevetti presi dallo scrape precedente
      print(link)
      wd.set_window_size(1920, 1080)
      wd.get(link)

      try:
        WebDriverWait(wd, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.registrazione"))
        )
      except:
        continue

      desc_tecnica = ''
      applicazioni = ''
      vantaggi = ''
      stato = ''
      data_priorita = ''
      licenza = ''
      diritti_comm = ''
      proprietari = ''
      contatti_proprietari = ''
      disponibilita = ''
      image_link = ''
      web_tags = ''

###  siccome alcuni brevetti non sono ancora disponibili, ho imposto una clausola else per evitarli ###

      if (len(wd.find_elements(By.CSS_SELECTOR,".corpobrevetto p")) > 0):
        intro = wd.find_elements(By.CSS_SELECTOR,".corpobrevetto p")[0].text
      else:
        print("Pagina anomala")
        anomalies.append(link)
        continue

      if (len(wd.find_elements(By.CSS_SELECTOR,".corpobrevetto p")) > 0):
        desc_tecnica = wd.find_elements(By.CSS_SELECTOR,".corpobrevetto p")[1].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".corpobrevetto ul")) > 1):
        applicazioni = wd.find_elements(By.CSS_SELECTOR,".corpobrevetto ul")[0].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".corpobrevetto ul")) > 1):
        vantaggi = wd.find_elements(By.CSS_SELECTOR,".corpobrevetto ul")[1].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")) > 1):
        stato = wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")[1].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")) > 3):
        data_priorita = wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")[3].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")) > 4):
        licenza = wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")[4].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")) > 5):
        diritti_comm = wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")[5].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".tagtag")) > 0):
        web_tags = wd.find_elements(By.CSS_SELECTOR,".tagtag")[0].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")) > 0):
        proprietari = wd.find_elements(By.CSS_SELECTOR,".infobrevetto span a")[0].text

      if (len(wd.find_elements(By.CSS_SELECTOR,".infobrevetto span a")) > 0):
        contatti_proprietari = wd.find_elements(By.CSS_SELECTOR,".infobrevetto span a")[0].get_attribute("href")

      if (len(wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")) > 6):
        disponibilita = wd.find_elements(By.CSS_SELECTOR,".infobrevetto span")[6].text

      if (len(wd.find_elements(By.CSS_SELECTOR, ".immaginebrevetto img")) > 0):
        image_link = wd.find_elements(By.CSS_SELECTOR,".immaginebrevetto img")[0].get_attribute("src")

      patent_details.append({'url': link,
                          'image_link': image_link,
                          'intro': intro,
                          'contatti_proprietari' : contatti_proprietari,
                          'applicazioni': applicazioni,
                          'vantaggi': vantaggi,
                          'stato': stato,
                          'web_tags': web_tags,
                          'desc_tecnica': desc_tecnica,
                          'licenza': licenza,
                          'diritti_comm': diritti_comm,
                          'data_priorita': data_priorita,
                          'proprietari': proprietari,
                          'disponibilita': disponibilita,
                          'title': patent['title'],
                          'description': patent['description']})

patent number 1/990
https://www.knowledge-share.eu/brevetto/biolab-perfusor-dispositivo-medico-di-perfusione-meccanica/
patent number 2/990
https://www.knowledge-share.eu/brevetto/a_atrofi-regolatori-della-massa-muscolare/
patent number 3/990
https://www.knowledge-share.eu/brevetto/health-met-casco-con-intelligenza-artificiale/
patent number 4/990
https://www.knowledge-share.eu/brevetto/3d-cell-migration-chipper-saggi-di-migrazione-cellulare/
Pagina anomala
patent number 5/990
https://www.knowledge-share.eu/brevetto/4842-geni-identificano-la-progenie-da-cd56-cd16perfnegcd7/
patent number 6/990
https://www.knowledge-share.eu/brevetto/acido-ialuronico-come-adiuvante-naturale-per-vaccini/
patent number 7/990
https://www.knowledge-share.eu/brevetto/acido-ialuronico-funzionalizzato/
patent number 8/990
https://www.knowledge-share.eu/brevetto/acl-quick-check-analisi-biomeccanica-del-rischio-di-infortunio-al-ginocchio/
patent number 9/990
https://www.knowledge-share.eu/brevetto/acquisizione-c

Sotto sono stati fatti alcuni controlli per funzionare che le pagine fossero state scrapate e che i dati fossero coerenti e completi

In [ ]:
print(len(patent_details))
print(len(anomalies))

In [ ]:
patent_details.info

<bound method DataFrame.info of                                                    url  \
0    https://www.knowledge-share.eu/brevetto/biolab...   
1    https://www.knowledge-share.eu/brevetto/a_atro...   
2    https://www.knowledge-share.eu/brevetto/health...   
3    https://www.knowledge-share.eu/brevetto/4842-g...   
4    https://www.knowledge-share.eu/brevetto/acido-...   
..                                                 ...   
930  https://www.knowledge-share.eu/brevetto/video-...   
931  https://www.knowledge-share.eu/brevetto/viruse...   
932  https://www.knowledge-share.eu/brevetto/visore...   
933  https://www.knowledge-share.eu/brevetto/weanca...   
934  https://www.knowledge-share.eu/brevetto/wound-...   

                                            image_link  \
0    https://www.knowledge-share.eu/wp-content/uplo...   
1    https://www.knowledge-share.eu/wp-content/uplo...   
2    https://www.knowledge-share.eu/wp-content/uplo...   
3    https://www.knowledge-share.eu/wp-

In [ ]:
import pandas as pd
patent_details = pd.DataFrame(patent_details)
patent_details.set_index("url")
patent_details.head()

,url,image_link,intro,contatti_proprietari,applicazioni,vantaggi,stato,web_tags,desc_tecnica,licenza,diritti_comm,data_priorita,proprietari,disponibilita,title,description
0,https://www.knowledge-share.eu/brevetto/biolab...,https://www.knowledge-share.eu/wp-content/uplo...,BioLab Perfusor è un dispositivo medico che pe...,https://www.knowledge-share.eu/proprietario/un...,Trapianto di organi e loro recupero marginale;...,Aumento organi disponibili per il trapianto gr...,Concesso,Medicina rigenerativaPerfusione ipotermicaPerf...,La perfusione meccanica viene utilizzata per m...,Italia,Esclusivi,06/11/2019,Università degli Studi di Firenze,Disponibile,‘BIOLAB PERFUSOR’ – Dispositivo medico di perf...,Permette di mantenere in condizioni fisiologic...
1,https://www.knowledge-share.eu/brevetto/a_atro...,https://www.knowledge-share.eu/wp-content/uplo...,Il brevetto verte sul possibile impiego di una...,https://www.knowledge-share.eu/proprietario/un...,"Nuovo trattamento farmacologico, senza equival...",Sostituzione totale o parziale degli integrato...,Depositato,Atrofia muscolareBV22CachessiaModulatoriPerdit...,Ad oggi non è disponibile un efficiente tratta...,Internazionale,Esclusivi,02/08/2019,Università degli Studi di Firenze,Disponibile,“A_Atrofi” – Regolatori della massa muscolare,"Miscela di glucocorticoidi, A_Atrofi, costitui..."
2,https://www.knowledge-share.eu/brevetto/health...,https://www.knowledge-share.eu/wp-content/uplo...,"L’invenzione, denomianta Health-Met, è una tec...",https://www.knowledge-share.eu/proprietario/un...,Sicurezza motociclisti;\nSicurezza sui luoghi ...,Riduzione dei tempi di soccorso;\nCondivisione...,Depositato,CascoInternet of Thingsprotezione delle person...,Health-Met è una tecnologia che ha l’obiettivo...,Internazionale,Esclusivi,26/05/2020,Università degli Studi di Firenze,Licenziato,“Health-Met” – Casco con Intelligenza Artificiale,Casco in grado di chiamare soccorsi in caso di...
3,https://www.knowledge-share.eu/brevetto/4842-g...,https://www.knowledge-share.eu/wp-content/uplo...,La presente invenzione riguarda la variazione ...,https://www.knowledge-share.eu/proprietario/un...,Attività anti-HCMV e anti-virale.\nFarmaci vol...,Differenziare la popolazione NKG2C+ matura dal...,Concesso,CD7CD94/NKG2CHCMVPerforinatrapianto di HSC,Il precursore CD56-CD16+PerfnegCD7– è stato id...,Italia,Esclusivi,13/12/2019,Università degli Studi di Genova,Disponibile,4842 geni identificano la progenie da CD56-CD1...,Variazione genetica denominata CD56-CD16+Perfn...
4,https://www.knowledge-share.eu/brevetto/acido-...,https://www.knowledge-share.eu/wp-content/uplo...,L’invenzione consiste nello sfruttare le propr...,https://www.knowledge-share.eu/proprietario/is...,Acido ialuronico usato come adiuvante immunolo...,Estrema efficacia nell’indurre una forte rispo...,Concesso,Acido IaluronicoAdiuvante immunologicoOncologi...,HA è un polimero intrinseco del nostro organis...,Internazionale,Esclusivi,26/10/2018,Istituto Oncologico Veneto,Disponibile,Acido Ialuronico come adiuvante naturale per v...,Innovativo adiuvante vaccinale in grado di ind...


In [ ]:
patent_details.to_csv("patent_details.csv")

# Immagini

Il prossimo passo sarà quello di scaricare le immagini utilizzando i link (image_link) raccolti nello scrape precedenti

In [ ]:
# open csv file
import pandas as pd
patent_details = pd.read_csv("patent_details.csv", index_col="Unnamed: 0")
patent_details.head()

,url,image_link,intro,contatti_proprietari,applicazioni,vantaggi,stato,web_tags,desc_tecnica,licenza,diritti_comm,data_priorita,proprietari,disponibilita,title,description
0,https://www.knowledge-share.eu/brevetto/biolab...,https://www.knowledge-share.eu/wp-content/uplo...,BioLab Perfusor è un dispositivo medico che pe...,https://www.knowledge-share.eu/proprietario/un...,Trapianto di organi e loro recupero marginale;...,Aumento organi disponibili per il trapianto gr...,Concesso,Medicina rigenerativaPerfusione ipotermicaPerf...,La perfusione meccanica viene utilizzata per m...,Italia,Esclusivi,06/11/2019,Università degli Studi di Firenze,Disponibile,‘BIOLAB PERFUSOR’ – Dispositivo medico di perf...,Permette di mantenere in condizioni fisiologic...
1,https://www.knowledge-share.eu/brevetto/a_atro...,https://www.knowledge-share.eu/wp-content/uplo...,Il brevetto verte sul possibile impiego di una...,https://www.knowledge-share.eu/proprietario/un...,"Nuovo trattamento farmacologico, senza equival...",Sostituzione totale o parziale degli integrato...,Depositato,Atrofia muscolareBV22CachessiaModulatoriPerdit...,Ad oggi non è disponibile un efficiente tratta...,Internazionale,Esclusivi,02/08/2019,Università degli Studi di Firenze,Disponibile,“A_Atrofi” – Regolatori della massa muscolare,"Miscela di glucocorticoidi, A_Atrofi, costitui..."
2,https://www.knowledge-share.eu/brevetto/health...,https://www.knowledge-share.eu/wp-content/uplo...,"L’invenzione, denomianta Health-Met, è una tec...",https://www.knowledge-share.eu/proprietario/un...,Sicurezza motociclisti;\nSicurezza sui luoghi ...,Riduzione dei tempi di soccorso;\nCondivisione...,Depositato,CascoInternet of Thingsprotezione delle person...,Health-Met è una tecnologia che ha l’obiettivo...,Internazionale,Esclusivi,26/05/2020,Università degli Studi di Firenze,Licenziato,“Health-Met” – Casco con Intelligenza Artificiale,Casco in grado di chiamare soccorsi in caso di...
3,https://www.knowledge-share.eu/brevetto/4842-g...,https://www.knowledge-share.eu/wp-content/uplo...,La presente invenzione riguarda la variazione ...,https://www.knowledge-share.eu/proprietario/un...,Attività anti-HCMV e anti-virale.\nFarmaci vol...,Differenziare la popolazione NKG2C+ matura dal...,Concesso,CD7CD94/NKG2CHCMVPerforinatrapianto di HSC,Il precursore CD56-CD16+PerfnegCD7– è stato id...,Italia,Esclusivi,13/12/2019,Università degli Studi di Genova,Disponibile,4842 geni identificano la progenie da CD56-CD1...,Variazione genetica denominata CD56-CD16+Perfn...
4,https://www.knowledge-share.eu/brevetto/acido-...,https://www.knowledge-share.eu/wp-content/uplo...,L’invenzione consiste nello sfruttare le propr...,https://www.knowledge-share.eu/proprietario/is...,Acido ialuronico usato come adiuvante immunolo...,Estrema efficacia nell’indurre una forte rispo...,Concesso,Acido IaluronicoAdiuvante immunologicoOncologi...,HA è un polimero intrinseco del nostro organis...,Internazionale,Esclusivi,26/10/2018,Istituto Oncologico Veneto,Disponibile,Acido Ialuronico come adiuvante naturale per v...,Innovativo adiuvante vaccinale in grado di ind...


In [ ]:
!mkdir images  #crea automaticamente una cartella chiamata "images"

Questo è il processo che scarica le immagini e successivamente le aggiunge alla cartella "images"

In [ ]:
import time
import requests

list_images = []
num = 0
for id, patent in tqdm(patent_details.iterrows(), total=patent_details.shape[0]):
  num = num + 1
  try:
    list_images.append({"url": patent["url"],
                        "img_file": f"img_{num}.jpg"})
    src = patent['image_link']
    img_file = requests.get(src, stream=True)
    if img_file.status_code == 200:
      with open("/content/images/img_" + str(num) + ".jpg", 'wb') as f:
        f.write(img_file.content)
  except Exception as e:
    print(e)



  0%|          | 0/935 [00:00<?, ?it/s]

Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?


Sotto andremo a fare dei controlli sui dati scrapati riguardo al numero e la qualità dell'immagine

In [ ]:
print(len(list_images))

935


In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/content/images/img_1.jpg')
imgplot = plt.imshow(img)
plt.show()

Qui viene fatta una visualizzazione delle immagini scrapate e poi salviamo tutto in formato csv e anche zip

In [ ]:
import pandas as pd
ds_images = pd.DataFrame(list_images)
ds_images.head()

,url,img_file
0,https://www.knowledge-share.eu/brevetto/biolab...,img_1.jpg
1,https://www.knowledge-share.eu/brevetto/a_atro...,img_2.jpg
2,https://www.knowledge-share.eu/brevetto/health...,img_3.jpg
3,https://www.knowledge-share.eu/brevetto/4842-g...,img_4.jpg
4,https://www.knowledge-share.eu/brevetto/acido-...,img_5.jpg


In [ ]:
ds_images.to_csv("ds_images.csv")

In [ ]:
!zip -r "/content/images.zip" "/content/images/"

# Tags

Quello seguente è un modello preso da Hugging Face già trainato a trasformare del testo in parole chiave, o come li abbiamo chiamati noi, tags

Al prodotto di questo modello andremo poi ad aggiungere tutte le informazioni della patent_details, essa contiene tutte le informazioni riguardo ai brevetti raccolte precedentemente dai vari scrapes

In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("efederici/text2tags")
tokenizer = AutoTokenizer.from_pretrained("efederici/text2tags")

def tag(text: str):
    """ Generates tags from given text """
    text = text.strip().replace('\n', '')
    text = 'summarize: ' + text
    tokenized_text = tokenizer.encode(text, return_tensors="pt")

    tags_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        max_length=20,
                                        early_stopping=True)

    output = tokenizer.decode(tags_ids[0], skip_special_tokens=True)
    return output.split(', ')

key = []

for patent_id, patent in patent_details.iterrows():
      desc_tecnica = patent["desc_tecnica"]   #link dei brevetti presi dallo scrape precedente
      tags = tag(desc_tecnica)
      print(tags)

      key.append({"tags" : tags,
                  'title': patent['title'],
                  'description': patent['description'],
                  "url" : patent["url"],
                  'introduzione' : patent['intro'],
                  'contatti_proprietari' : patent['contatti_proprietari'],
                  'applicazioni' : patent['applicazioni'],
                  'vantaggi' : patent['vantaggi'],
                  'stato' : patent['stato'],
                  'web_tags' : patent['web_tags'],
                  'desc_tecnica' : patent['desc_tecnica'],
                  'licenza' : patent['licenza'],
                  'diritti_comm' : patent['diritti_comm'],
                  'data_priorita' : patent['data_priorita'],
                  'proprietari' : patent['proprietari'],
                  'disponibilita' : patent['disponibilita']})

['perfusione meccanica', 'biolab perfusor']
['a_atrofi']
['health-met', 'caschi']
['cellule staminali', 'hcv']
['ha', 'proteina proteica', 'ha']
['ha', 'ha reticolato']
['infortunio legamento crociato anteriore', 'lca']
['immagini ecografiche', 'elettromiografia', 'sonda ecografica']
['adroterapico', 'pet']
['imaging di overhauser/pedri', 'spin probe']
['adenosina inalata', 'danno po']
['adesivi dentali', 'grafene']
['robot', 'terapista']
['adiuvanti', 'polimixina e1,']
['aequip']
['esogeni', 'cromofori/fluorocromi', '']
['leiomioma', 'anti-mirna']
['aon', 'oligonucleotidi']
['antagomirna']
['al.ma.', 'cancro colon-retto', 'ifobt']
['anamnestica', 'gps', 'trl 4']
['allineatore', 'allineatori dentali']
['summarize', 'wp']
['summarize', 'wp']
['emoglobina glicata', 'hba1c']
['agonisti trpv1']
['enea', 'lock-in amplifier', 'segnali deboli']
['fondoscopia', 'retinopatia ipertensiva', 'anomalie retinich']
['dls', 'cuvette']
['esami istologici intraoperatori', 'analisi tattili', 'biopsia,']


Soliti controlli per assicurarci della completezza e coerenza delle informazioni

In [ ]:
import pprint
pprint.pprint(key[0:5])

In [ ]:
import pandas as pd
v_tags = pd.DataFrame(key)
v_tags.set_index("url")
v_tags.head()

,tags,title,description,url,introduzione,contatti_proprietari,applicazioni,vantaggi,stato,web_tags,desc_tecnica,licenza,diritti_comm,data_priorita,proprietari,disponibilita
0,"[perfusione meccanica, biolab perfusor]",‘BIOLAB PERFUSOR’ – Dispositivo medico di perf...,Permette di mantenere in condizioni fisiologic...,https://www.knowledge-share.eu/brevetto/biolab...,BioLab Perfusor è un dispositivo medico che pe...,https://www.knowledge-share.eu/proprietario/un...,Trapianto di organi e loro recupero marginale;...,Aumento organi disponibili per il trapianto gr...,Concesso,Medicina rigenerativaPerfusione ipotermicaPerf...,La perfusione meccanica viene utilizzata per m...,Italia,Esclusivi,06/11/2019,Università degli Studi di Firenze,Disponibile
1,[a_atrofi],“A_Atrofi” – Regolatori della massa muscolare,"Miscela di glucocorticoidi, A_Atrofi, costitui...",https://www.knowledge-share.eu/brevetto/a_atro...,Il brevetto verte sul possibile impiego di una...,https://www.knowledge-share.eu/proprietario/un...,"Nuovo trattamento farmacologico, senza equival...",Sostituzione totale o parziale degli integrato...,Depositato,Atrofia muscolareBV22CachessiaModulatoriPerdit...,Ad oggi non è disponibile un efficiente tratta...,Internazionale,Esclusivi,02/08/2019,Università degli Studi di Firenze,Disponibile
2,"[health-met, caschi]",“Health-Met” – Casco con Intelligenza Artificiale,Casco in grado di chiamare soccorsi in caso di...,https://www.knowledge-share.eu/brevetto/health...,"L’invenzione, denomianta Health-Met, è una tec...",https://www.knowledge-share.eu/proprietario/un...,Sicurezza motociclisti;\nSicurezza sui luoghi ...,Riduzione dei tempi di soccorso;\nCondivisione...,Depositato,CascoInternet of Thingsprotezione delle person...,Health-Met è una tecnologia che ha l’obiettivo...,Internazionale,Esclusivi,26/05/2020,Università degli Studi di Firenze,Licenziato
3,"[cellule staminali, hcv]",4842 geni identificano la progenie da CD56-CD1...,Variazione genetica denominata CD56-CD16+Perfn...,https://www.knowledge-share.eu/brevetto/4842-g...,La presente invenzione riguarda la variazione ...,https://www.knowledge-share.eu/proprietario/un...,Attività anti-HCMV e anti-virale.\nFarmaci vol...,Differenziare la popolazione NKG2C+ matura dal...,Concesso,CD7CD94/NKG2CHCMVPerforinatrapianto di HSC,Il precursore CD56-CD16+PerfnegCD7– è stato id...,Italia,Esclusivi,13/12/2019,Università degli Studi di Genova,Disponibile
4,"[ha, proteina proteica, ha]",Acido Ialuronico come adiuvante naturale per v...,Innovativo adiuvante vaccinale in grado di ind...,https://www.knowledge-share.eu/brevetto/acido-...,L’invenzione consiste nello sfruttare le propr...,https://www.knowledge-share.eu/proprietario/is...,Acido ialuronico usato come adiuvante immunolo...,Estrema efficacia nell’indurre una forte rispo...,Concesso,Acido IaluronicoAdiuvante immunologicoOncologi...,HA è un polimero intrinseco del nostro organis...,Internazionale,Esclusivi,26/10/2018,Istituto Oncologico Veneto,Disponibile


E naturalmente, riportiamo tutto su csv.

In [ ]:
v_tags.to_csv("ds_tags.csv")

# The End